In [ ]:
import gym
import numpy as np

Vi må først sette opp et miljø. Gym kommer med et sett predefinerte miljøer, hvor 'FrozenLake-v0' er en av dem. Dette er et spill hvor en agent skal navigere en frossen insjø, som er representert med en 4x4 grid.

Eksempel:

SFFF<br>
FHFH<br>
FFFH<br>
HFFG<br>

S: start, trygg<br>
F: frossen overflate, trygg<br>
H: hull, ikke trygg<br>
G: mål, hvor agenten ønsker å komme<br>

En handling består i å bevege seg i en av fire retninger, men ettersom isen er glatt er det ikke garantert at man beveger seg i den retningen man ønsker.

In [ ]:
# Definer og sett opp miljøet
env = gym.make('FrozenLake-v0')
env.reset()

# env.action_space og env.obvservation_space definerer tilgjengelige handlinger og observasjoner
print("Action space:", env.action_space)
print("Observation space:", env.observation_space)

# vi kan visualisere et gitt miljø med env.render()
env.render()

<code>env.action_space</code> er altså lik <code>Discrete(4)</code>. Dette vil si at den kan ta fire ulike diskret verdier, som representerer et forsøk på å bevege seg i en av de fire retningene. <code>env.observation_space</code> er lik <code>Discrete(16)</code>, altså 16 ulike verdier som representerer de 4x4=16 mulige posisjonene agenten kan ha.

Det fins mange andre typer handlings- og tilstandsrom. Eksempelvis vil en handling i et handlingsrom lik <code>Box(4,)</code> være en array av fire tall

In [ ]:
# Simuler og visualiser fem tilfeldige handlinger
env.reset()
for _ in range(5):
    env.render()
    action = # Velg tilfeldig handling
    observation, reward, done, info = # Utfør handling

Som vi ser returnerer hvert steg fire verdier. De er som følger:

<code>observation</code> En verdi fra observasjonsrommet<br>
<code>reward</code> Et flyttall som representerer fortjenesten for agentens handling, i dette tilfellet 1.0 for målruten og 0.0 for alle andre<br>
<code>done</code> En boolean som forteller oss om spillet er fullført (enten fordi målet er nådd eller agenten har falt ned i et hull)<br>
<code>info</code> En dict med diverse debug-info

In [ ]:
print("observation:", observation)
print("reward:", reward)
print("done:", done)
print("info:", info)

Vi kan lære en strategi for dette problemet ved bruk av Q-learning algoritmen.

Epsilon-hyperparametrene under brukes for å kombinere Q-learning med simulated annealing: Vi velger
en tilfeldig handling med en viss sannsynlighet, hvor sannsynligheten for å velge den optimale verdien gitt Q-verditabellen øker med tid.

Nyttige funksjoner:

$\pi(s) = \arg\underset{a}{\max} Q(s,a)$

$Q^{new}(s_t, a_t) \leftarrow  Q(s_t, a_t) + \alpha*[r_t + \gamma * \underset{a}{\max} Q(s_{t+1},a) - Q(s_t, a_t)]$

In [ ]:
def learn_q(env,
            learning_rate = 0.7,
            gamma = 0.9,
            num_episodes = 20000,
            max_steps = 100,
            min_epsilon = 0.01,
            max_epsilon = 1.0,
            decay_rate = 0.005):
    Q = # Tom tabell med størrelse [size(observation_space) x size(action_space)]
    rewards = []

    epsilon = max_epsilon
    for episode in range(num_episodes):
        state = env.reset()
        reward_total = 0.0
        done = False
        for i in range(max_steps):
            tradeoff = np.random.uniform(0,1)

            # Velg en handling basert på Q-tabell eller utforsk ny handling
            if tradeoff > epsilon:
                action = # Velg beste handling gitt nåværende Q-tabell
            else:
                action = # Velg en tilfeldig handling

            state_new, reward, done, _ = # Utfør handling

            # Oppdater Q-verdi for tilstand-handling-par
            Q[state, action] = # Oppdateringsregel for Q-verdi
            
            # Oppdater total fortjeneste og tilstand
            reward_total += reward
            state = state_new

            # Avslutt dersom spill fullføres
            if done:
                break
        rewards.append(reward_total)
        epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
    final_reward = np.sum(rewards)/num_episodes
    return Q, final_reward

In [ ]:
Q, r = learn_q(env)
print("Endelig resultat:", r)
print("Endelige Q-verdier:", Q)

In [ ]:
# Test Q-tabell
state = env.reset()
done = False
while not done:
    env.render()
    action = # Velg optimal handling gitt Q-tabell
    state, reward, done, info = # Utfør handling

if(state == 15):
    print("Du klarte det! :D")
    env.render()
else:
    print("Du klarte det ikke :(")
    env.render()

Eksperimenter gjerne med ulike hyperparametre for å se om du kan forbedre resultatet!